In [ ]:
import numpy as np
from ml.algorithms.optimization import GradientDescent, StochasticGradientDescent
from ml.algorithms.normalization import MinMaxScaler
from ml.models.linear import PolynomialRegression, LinearRegression

In [ ]:
linear_weights_expected = np.array([[2.0, 3.0]])

x = np.array([
    [3.0], 
    [6.0], 
    [7.0]
])
y = np.array([
    [11.0], 
    [20.0], 
    [23.0]
])

epochs = 5000
learning_rate = 0.05
regularization = 0

In [ ]:
def polyreg(x, y, solver, epochs, learning_rate, regularization):
    regressor = PolynomialRegression(2)
    regressor.fit(x, y, solver, epochs, learning_rate, regularization, scaler=MinMaxScaler)

    print("Learned parameters: ", regressor.weights(0), "Expected: ", "??")


gd = GradientDescent()
polyreg(x, y, gd, epochs, learning_rate, regularization)

sgd = StochasticGradientDescent()
polyreg(x, y, sgd, epochs, learning_rate, regularization)

In [ ]:
def linreg(x, y, solver, epochs, learning_rate, regularization):
    regressor = LinearRegression()
    regressor.fit(x, y, solver, epochs, learning_rate, regularization)

    print("Learned parameters: ", regressor.weights(0), "Expected: ", linear_weights_expected)

gd = GradientDescent()
linreg(x, y, gd, epochs, learning_rate, regularization)